In [ ]:
!pip install -r requirements.txt --quiet

  Using cached numpy-1.20.0.zip (8.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [199 lines of output]
      setup.py:66: RuntimeWarning: NumPy 1.20.0 may not yet support Python 3.11.
        warnings.warn(
      Running from numpy source directory.
      setup.py:485: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
        run_build = parse_setuppy_commands()
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Processing numpy/random\_bounded_integers.pyx.in
      Processing numpy/random\_common.pyx
      Processing numpy/random\_generator.pyx
      Processing numpy/random\_mt19937.pyx
      Processing numpy/random\_pcg64.pyx
      Processing numpy/random\_philox.pyx
      Processing numpy/random\_sfc64.pyx
      blas_opt_info:
      blas_mkl_info:
   

In [2]:
DURATION = 0.006
BATCH_SIZE = 512

In [ ]:
import os
import wandb

# Optional: Set API Key via Environment Variable
os.environ["WANDB_API_KEY"] = "31da9995ef159ce5a9b8e9598918771ed1895164"
wandb.login()

# Initialize WandB Project
wandb.init(
    project="EDM Benchmarker",
    name="cifar10_exp_0",
    reinit=True  # Only if needed
)

wandb.config.update({
    "duration": DURATION,
    "batch_size": BATCH_SIZE,
})

# Logging Example Plot
wandb.log({"example_plot": 0})  # Placeholder value

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sleepyhunter (sleepyhunter-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DELL\_netrc


In [ ]:
import os
import json
from tqdm import tqdm
import torchvision
from torchvision.transforms import transforms
from PIL import Image

# Define paths for saving images and captions
output_dir = "./working/cifar10_stylegan2"
images_dir = os.path.join(output_dir, "images")
captions_dir = os.path.join(output_dir, "captions")

# Create directories
os.makedirs(images_dir, exist_ok=True)
os.makedirs(captions_dir, exist_ok=True)

# Load CIFAR-10 dataset (no preprocessing)
dataset = torchvision.datasets.CIFAR10(
    root="./input/data",  # Adjusted path for Kaggle
    train=True,
    transform=None,  # Keep images in original PIL format
    download=True
)

# Class names in CIFAR-10
class_names = dataset.classes

# Save images and captions
for idx, (image, label) in enumerate(tqdm(dataset, desc="Processing images")):
    # Save image
    image_path = os.path.join(images_dir, f"{idx:06d}.png")
    image.save(image_path)

    # Save caption as JSON
    caption_data = {"label": class_names[label], "index": idx}
    caption_path = os.path.join(captions_dir, f"{idx:06d}.json")
    with open(caption_path, "w") as f:
        json.dump(caption_data, f)

    if idx % 1000 == 0:
        print(f"Processed {idx}/{len(dataset)} images")


Files already downloaded and verified
Processed 0/50000 images
Processed 1000/50000 images
Processed 2000/50000 images
Processed 3000/50000 images
Processed 4000/50000 images
Processed 5000/50000 images
Processed 6000/50000 images
Processed 7000/50000 images
Processed 8000/50000 images
Processed 9000/50000 images
Processed 10000/50000 images
Processed 11000/50000 images
Processed 12000/50000 images
Processed 13000/50000 images
Processed 14000/50000 images
Processed 15000/50000 images
Processed 16000/50000 images
Processed 17000/50000 images
Processed 18000/50000 images
Processed 19000/50000 images
Processed 20000/50000 images
Processed 21000/50000 images
Processed 22000/50000 images
Processed 23000/50000 images
Processed 24000/50000 images
Processed 25000/50000 images
Processed 26000/50000 images
Processed 27000/50000 images
Processed 28000/50000 images
Processed 29000/50000 images
Processed 30000/50000 images
Processed 31000/50000 images
Processed 32000/50000 images
Processed 33000/50

In [ ]:
# Convert CIFAR-10 data into the required dataset format
!python dataset_tool.py --source="working/cifar10_stylegan2" --dest="working/datasets/cifar10-32x32.zip"

# Calculate FID reference statistics
!python fid.py ref --data="working/datasets/cifar10-32x32.zip" --dest="working/fid-refs/cifar10-32x32.npz"



100%|██████████| 50000/50000 [04:58<00:00, 167.31it/s]
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\distributed\distributed_c10d.py:613: UserWarning: Attempted to get default timeout for nccl backend, but NCCL support is not compiled
  warnings.warn("Attempted to get default timeout for nccl backend, but NCCL support is not compiled")
Traceback (most recent call last):
  File "c:\Users\DELL\OneDrive\Documents\GitHub\edm\fid.py", line 164, in <module>
    main()
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1078, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1688, in invoke
    return _process_result(sub_c

In [ ]:
!torchrun --nproc_per_node=1 train.py \
    --outdir="training-runs" \
    --data="datasets/cifar10-32x32.zip" \
    --cond=0 \
    --arch=ddpmpp \
    --batch=512 \
    --duration=1.5 \
    --tick=50 \
    --snap=2


W1217 01:47:11.223000 29352 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
[W socket.cpp:697] [c10d] The client socket has failed to connect to [DESKTOP-BHLTLKC]:29500 (system error: 10049 - The requested address is not valid in its context.).
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\distributed\distributed_c10d.py:613: UserWarning: Attempted to get default timeout for nccl backend, but NCCL support is not compiled
  warnings.warn("Attempted to get default timeout for nccl backend, but NCCL support is not compiled")
[W socket.cpp:697] [c10d] The client socket has failed to connect to [DESKTOP-BHLTLKC]:29500 (system error: 10049 - The requested address is not valid in its context.).
Traceback (most recent call last):
  File "c:\Users\DELL\OneDrive\Documents\GitHub\edm\train.py", line 234, in <module>
    main()
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-pack

In [ ]:
import os
import json

# Print the current working directory
print("Current working directory:", os.getcwd())

# File path to stats.jsonl
file_path = 'training-runs/00000-cifar10-32x32-uncond-ddpmpp-edm-gpus1-batch16-fp32/stats.jsonl'

# Check if the file exists
if not os.path.exists(file_path):
    print(f"Error: File not found at {file_path}")
    !ls training-runs  # Debugging: List available files
else:
    # Initialize a counter and storage for loss values
    loss_values = []

    # Open the .jsonl file and read line by line
    with open(file_path, 'r') as f:
        for line_number, line in enumerate(f, 1):  # Enumerate to track the line number
            try:
                # Parse the JSON object
                data = json.loads(line)

                # Extract 'Loss/loss' value if it exists
                if 'Loss/loss' in data and 'mean' in data['Loss/loss']:
                    loss_values.append(data['Loss/loss']['mean'])
                else:
                    print(f"No 'Loss/loss' key or 'mean' field on line {line_number}. Skipping...")

            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {line_number}. Skipping...")

    # Display the results
    print(f"\nTotal records processed: {line_number}")
    print(f"Total 'Loss/loss' values found: {len(loss_values)}")
    print("Sample Loss Values:")
    print("First 10:", loss_values[:10])
    print("Last 10:", loss_values[-10:])


Current working directory: c:\Users\DELL\OneDrive\Documents\GitHub\edm


E1217 01:47:16.291000 29352 torch\distributed\elastic\multiprocessing\api.py:826] failed (exitcode: 1) local_rank: 0 (pid: 16540) of binary: C:\Users\DELL\AppData\Local\Programs\Python\Python311\python.exe
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\DELL\AppData\Roaming\Python\Python311\Scripts\torchrun.exe\__main__.py", line 7, in <module>
  File "C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\distributed\elastic\multiprocessing\errors\__init__.py", line 347, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\distributed\run.py", line 879, in main
    run(args)
  File "C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\distributed\run.py", line 870, in run
    elastic_launch(
  File "C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\t

FileNotFoundError: [Errno 2] No such file or directory: 'training-runs/00000-cifar10-32x32-uncond-ddpmpp-edm-gpus1-batch16-fp32/stats.jsonl'

In [ ]:
import matplotlib.pyplot as plt

# Example array
data = loss_values

# Check for empty loss_values
if not data:
    print("No loss values to plot. Please check the input data.")
else:
    # Dynamic X-axis based on "epoch/50"
    x_axis = [i * 50 for i in range(len(data))]

    # Plotting
    plt.figure(figsize=(8, 5))
    plt.plot(x_axis, data, marker='o', linestyle='-', color='b', label='Loss')
    plt.title("Plot of Loss Values")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()

    # Save the plot as a file
    plt.savefig("loss_plot.png", dpi=300, bbox_inches='tight')
    print("Plot saved as 'loss_plot.png'")


In [ ]:
import os
import subprocess
import shutil
from tqdm import tqdm

res = []

# Define the folder path
folder_path = "training-runs/00000-cifar10-32x32-uncond-ddpmpp-edm-gpus1-batch16-fp32"

# List to store matching files
matching_files = []

# Step 1: Find matching files
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.startswith("n"):  # Check if the file name starts with "n"
            matching_files.append(file_name)
else:
    print(f"The folder '{folder_path}' does not exist or is not accessible.")

# Step 2: Process matching files
if matching_files:
    matching_files.sort()  # Sort files alphabetically for consistent order
    
    for filename in tqdm(matching_files, desc="Processing Models"):
        model_dir = os.path.join(folder_path, filename)
        
        # Clean up the output directory
        shutil.rmtree("out", ignore_errors=True)
        os.makedirs("out", exist_ok=True)
        
        # Run generate.py
        print(f"Generating images using model: {filename}")
        generate_cmd = [
            "python", "generate.py",
            "--outdir=out",
            "--seeds=0-63",
            "--batch=64",
            f"--network={model_dir}"
        ]
        subprocess.run(generate_cmd, check=True)
        
        # Run FID calculation
        print(f"Calculating FID for model: {filename}")
        fid_cmd = [
            "torchrun", "--standalone", "--nproc_per_node=1", "fid.py", "calc",
            "--images=out",
            "--num=64",
            "--ref=fid-refs/cifar10-32x32.npz"
        ]
        try:
            # Capture the output and get the last line (FID score)
            fid_output = subprocess.check_output(fid_cmd, text=True)
            last_line = fid_output.strip().split("\n")[-1]
            print(f"FID Score: {last_line}")
            res.append(float(last_line.split()[-1]))  # Assuming the FID is the last word
        except subprocess.CalledProcessError as e:
            print(f"Error calculating FID for model {filename}: {e}")
            res.append(None)

# Step 3: Display Results
print("\nFinal Results:")
for idx, score in enumerate(res):
    print(f"Model {matching_files[idx]}: FID = {score}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ensure 'data' is not empty
if not data:
    print("Error: No FID values to plot. Please check the input data.")
else:
    # Convert data to float (if not already)
    data = [float(i) for i in data]

    # Dynamic y-tick range based on data
    y_min, y_max = min(data), max(data)
    yticks = np.linspace(y_min, y_max, 11)  # Generate 11 evenly spaced ticks

    # Plot FID values
    plt.figure(figsize=(8, 5))
    plt.plot(range(len(data)), data, marker='o', linestyle='-', color='b', label='FID')
    plt.yticks(yticks, labels=[f"{tick:.3f}" for tick in yticks])  # Format ticks to 3 decimals
    plt.title("Plot of FID Values")
    plt.xlabel("Epoch / 50")
    plt.ylabel("FID")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # Save the plot
    output_path = "fid_values_plot.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Plot saved as '{output_path}'")

    # Show the plot
    plt.show()
